In [3]:
# ! pip install selenium beautifulsoup4 requests fake-useragent proxies


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 4.9 MB/s eta 0:00:00a 0:00:01
  Created wheel for proxies: filename=proxies-1.6-py3-none-any.whl size=2619 sha256=e73b7f9cacdb4ad794cce7f64ed129782980cee91ddb365a6bb98356d2dfff96
  Stored in directory: /Users/shubhamagarwal/Library/Caches/pip/wheels/c3/8c/ed/5e74fb4a655de4118d41cd27c45ab59fa851877fd7373ea635
Successfully built proxies
  Attempting uninstall: greenlet
    Found existing installation: greenlet 3.0.1
    Uninstalling greenlet-3.0.1:
      Successfully uninstalled greenlet-3.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.2.5 requires langchain-core<0.3.0,>=0.2.7, but you have langchain-core 0.3.19 which is incompatible.


# Version 1 of scraping step 1
- scrapes property url and property title

In [35]:
import time
import pandas as pd 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait

# Base URL
base_url = "https://www.apartments.com/houses/boston-ma/"

def get_driver():
  
    options = Options()
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.126 Safari/537.36"
    )
    options.add_argument("--start-maximized")  # Fullscreen mode
    
    driver = webdriver.Chrome(options=options)
    return driver

def wait_for_page_load(driver, timeout=15):

    WebDriverWait(driver, timeout).until(
        lambda d: d.execute_script('return document.readyState') == 'complete'
    )

def scrape_apartments(base_url, pages=11):

    all_properties = []
    
    for page in range(1, pages + 1):
        url = f"{base_url}{page}/"
        driver = get_driver()

        try:
            print(f"Scraping Page: {page} using your IP")
            driver.get(url)
            
            # Wait for the entire page to load
            wait_for_page_load(driver)
            
            for i in range(1, 41): 
                try:
                    xpath = f'/html/body/div[1]/main/section/div[1]/section[2]/div[2]/ul/li[{i}]/article'
                             /html/body/div[1]/main/section/div[1]/section[2]/div[2]/ul/li[40]/article/header/div[1]/a/div[2]
                    article = driver.find_element(By.XPATH, xpath)
                    
                    # Extract attributes
                    data_url = article.get_attribute("data-url")
                    data_streetaddress = article.get_attribute("data-streetaddress")
                    
                    # Append to results if data is found
                    if data_url and data_streetaddress:
                        property_info = {
                            "property_url": data_url,  # Renamed for clarity
                            "property_title": data_streetaddress  # Renamed for clarity
                        }
                        all_properties.append(property_info)
                        
                        # Print the scraped info
                        print(f"Property {len(all_properties)}: {property_info}")
                
                except Exception as e:
                    print(f"Error processing article {i} on page {page}: {e}")
            
            # Wait 3 seconds after scraping the page
            time.sleep(5)
            
        except Exception as e:
            print(f"Error on Page {page}: {e}")
        finally:
            # Close the browser
            driver.quit()

    return all_properties

def save_to_csv(data, filename="properties.csv"):

    # Convert list of dictionaries to a pandas DataFrame
    df = pd.DataFrame(data)
    # Save DataFrame to CSV
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

# use the function
properties = scrape_apartments(base_url, pages=11)
print(f"\nScraped {len(properties)} properties in total.")

# saving data
save_to_csv(properties, filename="properties.csv")


Scraping Page: 1 using your IP
Property 1: {'property_url': 'https://www.apartments.com/482-columbus-ave-boston-ma/d9qlk3h/', 'property_title': '482 Columbus Ave'}
Property 2: {'property_url': 'https://www.apartments.com/29-woodford-st-boston-ma/3vzh1xf/', 'property_title': '29 Woodford St'}
Property 3: {'property_url': 'https://www.apartments.com/368-e-8th-st-boston-ma/3msc2nd/', 'property_title': '368 E 8th St'}
Property 4: {'property_url': 'https://www.apartments.com/12-dalrymple-st-boston-ma/1xexzd5/', 'property_title': '12 Dalrymple St'}
Property 5: {'property_url': 'https://www.apartments.com/73-langley-rd-brighton-ma/zlt8enw/', 'property_title': '73 Langley Rd'}
Property 6: {'property_url': 'https://www.apartments.com/245-river-st-cambridge-ma/0eqcq52/', 'property_title': '245 River St'}
Property 7: {'property_url': 'https://www.apartments.com/51-siren-st-winthrop-ma/mh7fngh/', 'property_title': '51 Siren St'}
Property 8: {'property_url': 'https://www.apartments.com/8-jefferson-

# Version 2 of step 1 scraping script.
- scrapes the property url, property title(address as just the house number and street name), and the complete property address.

In [115]:
import time
import pandas as pd  # For saving data to CSV
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait

# Base URL
base_url = "https://www.apartments.com/houses/boston-ma/"

def get_driver():
 
    options = Options()
    # Mimic a standard browser user-agent
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.126 Safari/537.36"
    )
    options.add_argument("--start-maximized")  # Fullscreen mode
    
    driver = webdriver.Chrome(options=options)
    return driver

def wait_for_page_load(driver, timeout=15):
   
    WebDriverWait(driver, timeout).until(
        lambda d: d.execute_script('return document.readyState') == 'complete'
    )

def scrape_apartments(base_url, pages=11):

    all_properties = []
    
    for page in range(1, pages + 1):
        url = f"{base_url}{page}/"
        driver = get_driver()

        try:
            print(f"Scraping Page: {page} using your IP")
            driver.get(url)
            
            # Wait for the entire page to load
            wait_for_page_load(driver)
            
            # Loop through articles (1 to 40) using full XPath
            for i in range(1, 41):  # Assuming 40 articles per page
                try:
                    article_xpath = f'/html/body/div[1]/main/section/div[1]/section[2]/div[2]/ul/li[{i}]/article'
                    address_xpath = article_xpath + '/header/div[1]/a/div[2]'
                    
                    # Extract property title (street address)
                    try:
                        article_element = driver.find_element(By.XPATH, article_xpath)
                        property_url = article_element.get_attribute("data-url")
                        property_title = article_element.get_attribute("data-streetaddress")
                    except Exception as e:
                        property_url = None
                        property_title = None
                        print(f"Error getting title or URL for article {i} on page {page}: {e}")
                    
                    # Extract property address
                    try:
                        address_element = driver.find_element(By.XPATH, address_xpath)
                        property_address = address_element.text.strip()
                    except Exception as e:
                        property_address = None
                        print(f"Error getting address for article {i} on page {page}: {e}")
                    
                    # Append to results if data is found
                    if property_url and property_title:
                        property_info = {
                            "property_url": property_url,
                            "property_title": property_title,
                            "property_address": property_address
                        }
                        all_properties.append(property_info)
                        
                        # Print the scraped info
                        print(f"Property {len(all_properties)}: {property_info}")
                
                except Exception as e:
                    print(f"Error processing article {i} on page {page}: {e}")
            
            # Wait 5 seconds after scraping the page
            time.sleep(5)
            
        except Exception as e:
            print(f"Error on Page {page}: {e}")
        finally:
            # Close the browser
            driver.quit()

    return all_properties

def save_to_csv(data, filename="properties.csv"):

    # Convert list of dictionaries to a pandas DataFrame
    df = pd.DataFrame(data)
    # Save DataFrame to CSV
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

# Example usage
properties = scrape_apartments(base_url, pages=11)
print(f"\nScraped {len(properties)} properties in total.")

# Save data to CSV
save_to_csv(properties, filename="properties.csv")


Scraping Page: 1 using your IP
Property 1: {'property_url': 'https://www.apartments.com/29-woodford-st-boston-ma/3vzh1xf/', 'property_title': '29 Woodford St', 'property_address': '29 Woodford St, Boston, MA 02125'}
Property 2: {'property_url': 'https://www.apartments.com/368-e-8th-st-boston-ma/3msc2nd/', 'property_title': '368 E 8th St', 'property_address': '368 E 8th St, Boston, MA 02127'}
Property 3: {'property_url': 'https://www.apartments.com/12-dalrymple-st-boston-ma/1xexzd5/', 'property_title': '12 Dalrymple St', 'property_address': '12 Dalrymple St, Boston, MA 02130'}
Property 4: {'property_url': 'https://www.apartments.com/482-columbus-ave-boston-ma/d9qlk3h/', 'property_title': '482 Columbus Ave', 'property_address': '482 Columbus Ave, Boston, MA 02118'}
Property 5: {'property_url': 'https://www.apartments.com/245-river-st-cambridge-ma/0eqcq52/', 'property_title': '245 River St', 'property_address': '245 River St, Cambridge, MA 02139'}
Property 6: {'property_url': 'https://www

# Step 2 of scraping Housing Data
- this imports the csv file from step 1 and goes to each of the hrefs/urls to get specific page information.
- For Instance: Images(5), Monthly Rent, Bedrooms, Bathrooms, Square Feet and more.

In [171]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

def get_driver():

    options = Options()
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.126 Safari/537.36"
    )
    options.add_argument("--start-maximized")  # Start browser maximized
    driver = webdriver.Chrome(options=options)
    return driver

def extract_image_url(style_attribute):

    if not style_attribute:
        return None
    try:
        start_idx = style_attribute.find("url(") + 4
        end_idx = style_attribute.find(")", start_idx)
        image_url = style_attribute[start_idx:end_idx].strip('"')
        return image_url
    except Exception as e:
        print(f"Error extracting image URL: {e}")
        return None



def scrape_property_details(csv_file, start_row=0, end_row=None):

    # Read the input CSV
    properties = pd.read_csv(csv_file)
    
    # Apply slicing based on start_row and end_row
    properties = properties.iloc[start_row:end_row]
    
    scraped_data = []
    driver = get_driver()



    
    for index, row in properties.iterrows():
        property_title = row["property_title"]
        property_url = row["property_url"]
        property_address = row["property_address"]
        
        print(f"Scraping: {property_title} ({property_url})")
        
        # Initialize placeholders
        picture_1, picture_2, picture_3, picture_4, picture_5 = None, None, None, None, None
        monthly_rent, bedrooms, bathrooms, square_feet = None, None, None, None
        house_feature_1, house_feature_2, house_feature_3, house_feature_4, house_feature_5 = None, None, None, None, None
        neighborhood_p1, neighborhood_p2, neighborhood_p3 = None, None, None
        about_property_p1, about_property_p2 = None, None
        
        try:
            driver.get(property_url)
            time.sleep(5)  # Wait for the page to load
            
            # # Scrape images
            # try:
            #     xpath = '//*[@id="carouselSection"]/div[2]/div/ul/li[1]/div/div/div/div'
            #     element = driver.find_element(By.XPATH, xpath)
            #     style_attribute = element.get_attribute("style")
            #     picture_1 = extract_image_url(style_attribute)
            #     print(f"Found picture_1: {picture_1}")
            # except Exception as e:
            #     print(f"Could not get picture_1: {e}")

            # Scrape images
            try:
                # Primary XPath for picture_1
                xpath = "//*[@id='carouselSection']/div[2]/div/ul/li[1]/div/div/div/div"
                element = driver.find_element(By.XPATH, xpath)
                style_attribute = element.get_attribute("style")
                picture_1 = extract_image_url(style_attribute)
                print(f"Found picture_1: {picture_1}")
            except Exception as e:
                print(f"Primary XPath could not get picture_1")
                print("\n")
                
                # Fallback XPath for picture_1
                try:
                    fallback_xpath = "//*[@id='carouselSection']/div[1]/div/ul/li/div/div/div/div"
                    element = driver.find_element(By.XPATH, fallback_xpath)
                    style_attribute = element.get_attribute("style")
                    picture_1 = extract_image_url(style_attribute)
                    print(f"Found picture_1 with fallback XPath: {picture_1}")
                except Exception as e:
                    print(f"Fallback XPath also could not get picture_1")
                    print("\n")

            
            try:
                xpath = "//*[@id='carouselSection']/div[2]/div/ul/li[2]/div[1]/div/div/div"
                element = driver.find_element(By.XPATH, xpath)
                style_attribute = element.get_attribute("style")
                picture_2 = extract_image_url(style_attribute)
                print(f"Found picture_2: {picture_2}")
            except Exception as e:
                print(f"Could not get picture_2")
                print("\n")
            
            try:
                xpath = "//*[@id='carouselSection']/div[2]/div/ul/li[2]/div[2]/div/div/div"
                element = driver.find_element(By.XPATH, xpath)
                style_attribute = element.get_attribute("style")
                picture_3 = extract_image_url(style_attribute)
                print(f"Found picture_3: {picture_3}")
            except Exception as e:
                print(f"Could not get picture_3")
                print("\n")
            
            try:
                xpath = "//*[@id='carouselSection']/div[2]/div/ul/li[3]/div[1]/div/div/div"
                element = driver.find_element(By.XPATH, xpath)
                style_attribute = element.get_attribute("style")
                picture_4 = extract_image_url(style_attribute)
                print(f"Found picture_4: {picture_4}")
            except Exception as e:
                print(f"Could not get picture_4")
                print("\n")
            
            try:
                xpath = "//*[@id='carouselSection']/div[2]/div/ul/li[3]/div[2]/div/div/div"
                element = driver.find_element(By.XPATH, xpath)
                style_attribute = element.get_attribute("style")
                picture_5 = extract_image_url(style_attribute)
                print(f"Found picture_5: {picture_5}")
            except Exception as e:
                print(f"Could not get picture_5")
                print("\n")
            
            # Scrape additional details
            try:
                monthly_rent = driver.find_element(
                    By.XPATH, "//*[@id='priceBedBathAreaInfoWrapper']/div/div/ul/li[1]/div/p[2]"
                ).text
                print(f"Monthly Rent: {monthly_rent}")
            except Exception as e:
                print(f"Could not get monthly_rent")
                monthly_rent = None
                print("\n")
            
            try:
                bedrooms = driver.find_element(
                    By.XPATH, "//*[@id='priceBedBathAreaInfoWrapper']/div/div/ul/li[2]/div/p[2]"
                ).text
                print(f"Bedrooms: {bedrooms}")
            except Exception as e:
                print(f"Could not get bedrooms")
                bedrooms = None
                print("\n")
            
            try:
                bathrooms = driver.find_element(
                    By.XPATH, "//*[@id='priceBedBathAreaInfoWrapper']/div/div/ul/li[3]/div/p[2]"
                ).text
                print(f"Bathrooms: {bathrooms}")
            except Exception as e:
                print(f"Could not get bathrooms")
                bathrooms = None
                print("\n")
            
            try:
                square_feet = driver.find_element(
                    By.XPATH, "//*[@id='priceBedBathAreaInfoWrapper']/div/div/ul/li[4]/div/p[2]"
                ).text
                print(f"Square Feet: {square_feet}")
            except Exception as e:
                print(f"Could not get square_feet")
                square_feet = None
                print("\n")


            try:
                availability = driver.find_element(
                    By.XPATH, "//*[@id='unitDetailInfoWrapper']/div[1]/span/span[3]"
                ).text
                print(f"Availability: {availability}")
            except Exception as e:
                print(f"Could not get availability")
                availability = None
                print("\n")


            try:
                deposit = driver.find_element(
                    By.XPATH, "//*[@id='unitDetailInfoWrapper']/div[1]/span/span[2]"
                ).text
                print(f"Deposit: {deposit}")
            except Exception as e:
                print(f"Could not get deposit")
                deposit = None
                print("\n")

            try:
                lease_duration = driver.find_element(
                    By.XPATH, "//*[@id='unitDetailInfoWrapper']/div[1]/span/span[1]"
                ).text
                print(f"Lease Duration: {lease_duration}")
            except Exception as e:
                print(f"Could not get lease_duration")
                lease_duration = None
                print("\n")

            try:
                about_property_p1 = driver.find_element(
                    By.XPATH, "//*[@id='descriptionSection']/p[1]"
                ).text
                print(f"About Property (Paragraph 1): {about_property_p1}")
            except Exception as e:
                print(f"Could not get about_property_p1")
                about_property_p1 = None
                print("\n")

            try:
                about_property_p2 = driver.find_element(
                    By.XPATH, "//*[@id='descriptionSection']/p[2]"
                ).text
                print(f"About Property (Paragraph 2): {about_property_p2}")
            except Exception as e:
                print(f"Could not get about_property_p2")
                about_property_p2 = None
                print("\n")

            # Create a single string combining all about property paragraphs
            about_property_details = " ".join(
                filter(None, [about_property_p1, about_property_p2])
            )
            print(f"About Property Details: {about_property_details}")



            try:
                house_feature_1 = driver.find_element(
                    By.XPATH, '//*[@id="amenitiesSection"]/div/div/div/div/div/ul/li[1]/span'
                ).text
                print(f"House Feature 1: {house_feature_1}")
            except Exception as e:
                print(f"Could not get house_feature_1")
                house_feature_1 = None
                print("\n")


            try:
                house_feature_2 = driver.find_element(
                    By.XPATH, '//*[@id="amenitiesSection"]/div/div/div/div/div/ul/li[2]/span'
                ).text
                print(f"House Feature 2: {house_feature_2}")
            except Exception as e:
                print(f"Could not get house_feature_2")
                house_feature_2 = None
                print("\n")


            try:
                house_feature_3 = driver.find_element(
                    By.XPATH, '//*[@id="amenitiesSection"]/div/div[2]/div/div/div/ul/li[3]/span'
                ).text
                print(f"House Feature 3: {house_feature_3}")
            except Exception as e:
                print(f"Could not get house_feature_3")
                house_feature_3 = None
                print("\n")


            try:
                house_feature_4 = driver.find_element(
                    By.XPATH, '//*[@id="amenitiesSection"]/div/div[2]/div/div/div/ul/li[4]/span'
                ).text
                print(f"House Feature 4: {house_feature_4}")
            except Exception as e:
                print(f"Could not get house_feature_4")
                house_feature_4 = None
                print("\n")


            
            try:
                house_feature_5 = driver.find_element(
                    By.XPATH, '//*[@id="amenitiesSection"]/div/div[2]/div/div/div/ul/li[5]/span'
                ).text
                print(f"House Feature 5: {house_feature_5}")
            except Exception as e:
                print(f"Could not get house_feature_5")
                house_feature_5 = None
                print("\n")

            # House feature concat block
            # Create a comma-separated string of all house features
            house_features = ", ".join(
                filter(None, [house_feature_1, house_feature_2, house_feature_3, house_feature_4, house_feature_5])
            )
            print(f"All House Features: {house_features}")


            try:
                neighborhood_p1 = driver.find_element(
                    By.XPATH, '//*[@id="subMarketSection"]/div/div/div/div/p[2]'
                ).text
                print(f"Neighborhood Paragraph 1: {neighborhood_p1}")
            except Exception as e:
                print(f"Could not get neighborhood_p1")
                neighborhood_p1 = None
                print("\n")

            
            try:
                neighborhood_p2 = driver.find_element(
                    By.XPATH, '//*[@id="subMarketSection"]/div/div/div/div/p[3]'
                ).text
                print(f"Neighborhood Paragraph 2: {neighborhood_p2}")
            except Exception as e:
                print(f"Could not get neighborhood_p2")
                neighborhood_p2 = None
                print("\n")


            try:
                neighborhood_p3 = driver.find_element(
                    By.XPATH, '//*[@id="subMarketSection"]/div/div/div/div/p[4]'
                ).text
                print(f"Neighborhood Paragraph 3: {neighborhood_p3}")
            except Exception as e:
                print(f"Could not get neighborhood_p3")
                neighborhood_p3 = None
                print("\n")



            # Neighborhood concat block
            # Create a single string combining all neighborhood paragraphs
            neighborhood_details = " ".join(
                filter(None, [neighborhood_p1, neighborhood_p2, neighborhood_p3])
            )
            print(f"All Neighborhood Details: {neighborhood_details}")



            try:
                utilities_included_1 = driver.find_element(
                    By.XPATH, "//*[@id='profileV2FeesWrapper']/div[2]/div[1]/div/div/div[2]/ul/li[1]/div/div"
                ).text
                print(f"Utilities Included 1: {utilities_included_1}")
            except Exception as e:
                print(f"Could not get utilities_included_1")
                utilities_included_1 = None
                print("\n")

            try:
                utilities_included_2 = driver.find_element(
                    By.XPATH, "//*[@id='profileV2FeesWrapper']/div[2]/div[1]/div/div/div[2]/ul/li[2]/div/div"
                ).text
                print(f"Utilities Included 2: {utilities_included_2}")
            except Exception as e:
                print(f"Could not get utilities_included_2")
                utilities_included_2 = None
                print("\n")


            try:
                utilities_included_3 = driver.find_element(
                    By.XPATH, "//*[@id='profileV2FeesWrapper']/div[2]/div[1]/div/div/div[2]/ul/li[3]/div/div"
                ).text
                print(f"Utilities Included 3: {utilities_included_3}")
            except Exception as e:
                # print(f"Could not get utilities_included_3: {e}")
                print(f"Could not get utilities_included_3:")
                utilities_included_3 = None
                print("\n")

            try:
                utilities_included_4 = driver.find_element(
                    By.XPATH, "//*[@id='profileV2FeesWrapper']/div[2]/div[1]/div/div/div[2]/ul/li[4]/div/div"
                ).text
                print(f"Utilities Included 4: {utilities_included_4}")
            except Exception as e:
                print(f"Could not get utilities_included_4:")
                utilities_included_4 = None
                print("\n")

            # Create a comma-separated string combining all utilities included
            utilities_included = ", ".join(
                filter(None, [utilities_included_1, utilities_included_2, utilities_included_3, utilities_included_4])
            )
            print(f"All Utilities Included: {utilities_included}")





        
        except Exception as e:
            print(f"Error scraping {property_title}: {e}")


        scraped_data.append({
            "property_title": property_title,
            "property_address": property_address,
            "property_url": property_url,
            "monthly_rent": monthly_rent,
            "bedrooms": bedrooms,
            "bathrooms": bathrooms,
            "square_feet": square_feet,
            "availability": availability,
            "deposit": deposit,
            "lease_duration": lease_duration,
            "about_property_p1": about_property_p1,
            "about_property_p2": about_property_p2,
            "about_property_details": about_property_details,
            "house_feature_1": house_feature_1,
            "house_feature_2": house_feature_2,
            "house_feature_3": house_feature_3,
            "house_feature_4": house_feature_4,
            "house_feature_5": house_feature_5,
            "house_features": house_features,
            "neighborhood_p1": neighborhood_p1,
            "neighborhood_p2": neighborhood_p2,
            "neighborhood_p3": neighborhood_p3,
            "neighborhood_details": neighborhood_details,
            "utilities_included_1": utilities_included_1,
            "utilities_included_2": utilities_included_2,
            "utilities_included_3": utilities_included_3,
            "utilities_included_4": utilities_included_4,
            "utilities_included": utilities_included,
            "picture_1": picture_1,
            "picture_2": picture_2,
            "picture_3": picture_3,
            "picture_4": picture_4,
            "picture_5": picture_5,
        })

    driver.quit()
    
    # Create a DataFrame from the scraped data
    df = pd.DataFrame(scraped_data)
    return df

# Example usage
csv_file = "properties_stage_1.csv"
output_file = "properties_step2/properties_300_to_400_step2.csv"

df = scrape_property_details(csv_file, start_row=300, end_row=400) 
# print(df)

# Save to CSV
df.to_csv(output_file, index=False)
print(f"Data saved to {output_file}")


Scraping: 14 Linnaean St (https://www.apartments.com/14-linnaean-st-cambridge-ma/bv11n59/)
Found picture_1: https://images1.apartments.com/i2/4sDCwXorCzdYw4pDw51tG5e3JoCUJibeqhrokqNKqRY/111/14-linnaean-st-cambridge-ma-primary-photo.jpg?p=1
Found picture_2: https://images1.apartments.com/i2/sy5HLxXdzBrJcoFGpESTAB83E0qTeuRLDuqfyB4SO9s/117/14-linnaean-st-cambridge-ma-building-photo.jpg?p=1
Found picture_3: https://images1.apartments.com/i2/C5Ki7zDj_UJYiVk75dDPZz9FwqBqvXXrF0ioEktJypk/117/14-linnaean-st-cambridge-ma-building-photo.jpg?p=1
Found picture_4: https://images1.apartments.com/i2/uGlPX7CTegmxsi6jq6gND2CSK4Vd9Yi438nO-9IdyUI/117/14-linnaean-st-cambridge-ma-building-photo.jpg?p=1
Found picture_5: https://images1.apartments.com/i2/8yQH4OLUmFRFGSbmj5YluH8jnz3hk9wfgA1CKIDiuq8/117/14-linnaean-st-cambridge-ma-building-photo.jpg?p=1
Monthly Rent: $8,000
Bedrooms: 4 bd
Bathrooms: 4.5 ba
Square Feet: 2,796 sq ft
Availability: Available Now
Deposit: $8,000 deposit
Lease Duration: 
About Proper

# Step 3 of scraping images

In [6]:
# csv being used

merged_properties_with_pids = pd.read_csv("/Users/shubhamagarwal/Documents/Northeastern/Semester_3/final_project/Data/properties_step2/added_propertyid/merged_properties.csv")


In [8]:
merged_properties_with_pids.head()

,property_id,property_title,property_address,property_url,monthly_rent,bedrooms,bathrooms,square_feet,availability,deposit,...,utilities_included_1,utilities_included_2,utilities_included_3,utilities_included_4,utilities_included,picture_1,picture_2,picture_3,picture_4,picture_5
0,property_1,29 Woodford St,"29 Woodford St, Boston, MA 02125",https://www.apartments.com/29-woodford-st-bost...,"$5,000",4 bd,3 ba,"2,000 sq ft",Available Now,NaN,...,NaN,NaN,NaN,NaN,NaN,https://images1.apartments.com/i2/j5RbiHRTPlwj...,https://images1.apartments.com/m2/pYVHN7OSdT8U...,https://images1.apartments.com/i2/IOT5m2yQNBAA...,https://images1.apartments.com/i2/y80lXN-UIQaC...,https://images1.apartments.com/i2/aJaHwJO9p4gH...
1,property_2,368 E 8th St,"368 E 8th St, Boston, MA 02127",https://www.apartments.com/368-e-8th-st-boston...,"$3,300",2 bd,2 ba,"1,327 sq ft",Available Now,NaN,...,NaN,NaN,NaN,NaN,NaN,https://images1.apartments.com/i2/LWDxvKBwOyCU...,https://images1.apartments.com/m2/4UATYxV2VwYW...,https://images1.apartments.com/i2/8-t5Vi55lk9g...,https://images1.apartments.com/i2/Sv47btzME27j...,https://images1.apartments.com/i2/1Vu08j04n2NM...
2,property_3,12 Dalrymple St,"12 Dalrymple St, Boston, MA 02130",https://www.apartments.com/12-dalrymple-st-bos...,"$4,450",5 bd,2 ba,"1,900 sq ft",Available Now,NaN,...,NaN,NaN,NaN,NaN,NaN,https://images1.apartments.com/i2/HligSx7ii-kI...,https://images1.apartments.com/m2/lnu38brKzSry...,https://images1.apartments.com/i2/v-gpZtt9WPJY...,https://images1.apartments.com/i2/nVZF2Gt8JV5M...,https://images1.apartments.com/i2/rY556DgbSBrS...
3,property_4,482 Columbus Ave,"482 Columbus Ave, Boston, MA 02118",https://www.apartments.com/482-columbus-ave-bo...,"$4,400",2 bd,1 ba,"1,000 sq ft",Available Now,NaN,...,NaN,NaN,NaN,NaN,NaN,https://images1.apartments.com/i2/Ivwe9i-veGbL...,https://images1.apartments.com/m2/5hwAOflmUT5K...,https://images1.apartments.com/i2/HegsUU735WrS...,https://images1.apartments.com/i2/9UD-ha1NNydn...,https://images1.apartments.com/i2/JOgr4o5xf5uA...
4,property_5,245 River St,"245 River St, Cambridge, MA 02139",https://www.apartments.com/245-river-st-cambri...,"$5,900",3 bd,2 ba,"1,470 sq ft",Available Now,NaN,...,NaN,NaN,NaN,NaN,NaN,https://images1.apartments.com/i2/S1YRxOBS6chm...,https://images1.apartments.com/m2/oU1tUZ-JFgXD...,https://images1.apartments.com/i2/F8fOj4etww8S...,https://images1.apartments.com/i2/oCJ57dtxnE1b...,https://images1.apartments.com/i2/ppBzIgWGK6YW...


In [10]:
merged_properties_with_pids.columns 

Index(['property_id', 'property_title', 'property_address', 'property_url',
       'monthly_rent', 'bedrooms', 'bathrooms', 'square_feet', 'availability',
       'deposit', 'lease_duration', 'about_property_p1', 'about_property_p2',
       'about_property_details', 'house_feature_1', 'house_feature_2',
       'house_feature_3', 'house_feature_4', 'house_feature_5',
       'house_features', 'neighborhood_p1', 'neighborhood_p2',
       'neighborhood_p3', 'neighborhood_details', 'utilities_included_1',
       'utilities_included_2', 'utilities_included_3', 'utilities_included_4',
       'utilities_included', 'picture_1', 'picture_2', 'picture_3',
       'picture_4', 'picture_5'],
      dtype='object')

In [ ]:
input_csv = "/Users/shubhamagarwal/Documents/Northeastern/Semester_3/final_project/Data/properties_step2/added_propertyid/merged_properties.csv"  # Replace with the path to your updated CSV
output_csv = "/Users/shubhamagarwal/Documents/Northeastern/Semester_3/final_project/Data/properties_step3/properties_step3.csv"  # Replace with the path to save the final CSV
output_folder = "/Users/shubhamagarwal/Documents/Northeastern/Semester_3/final_project/Data/properties_step3/property_images"     # Folder to save all property images


In [63]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from PIL import Image


def get_driver(user_agent):
    """
    Set up and return a Selenium WebDriver instance with a specific user agent.
    """
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--start-maximized")  # Start the browser maximized
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument(f"user-agent={user_agent}")  # Use the provided user agent
    driver = webdriver.Chrome(options=options)
    return driver


def crop_screenshot(screenshot_path, cropped_path):

    try:
        with Image.open(screenshot_path) as img:
            width, height = img.size
            # Define cropping region (center 50% width and 60% height)
            left = width * 0.25
            right = width * 0.75
            top = height * 0.2
            bottom = height * 0.8

            cropped_img = img.crop((left, top, right, bottom))
            cropped_img.save(cropped_path)
            print(f"Cropped and saved image to {cropped_path}")
    except Exception as e:
        print(f"Error cropping image {screenshot_path}: {e}")


def open_images_in_browser(input_csv, output_csv, output_folder, user_agent):

    # Read the input CSV
    df = pd.read_csv(input_csv)

    # Add a column for image folder paths
    df['image_folder_path'] = None

    # Initialize Selenium WebDriver
    driver = get_driver(user_agent)

    # Iterate over each row
    for index, row in df.iterrows():
        property_id = row['property_id']
        property_folder = os.path.join(output_folder, property_id)

        # Create the folder only if an image is processed
        image_processed = False

        for i in range(1, 6):  # Loop through picture_1 to picture_5
            picture_column = f'picture_{i}'
            image_url = row[picture_column]

            if pd.notna(image_url):  # Check if the URL is valid
                try:
                    # Open the image URL in Chrome
                    driver.get(image_url)
                    # time.sleep(3)  # Wait for the page to load
                    # adding random wait instead of fixed 3 seconds wait to replicate human like behaviour
                    time.sleep(random.uniform(1, 2))

                    # Create the folder only when processing the first valid image
                    if not image_processed:
                        os.makedirs(property_folder, exist_ok=True)
                        df.at[index, 'image_folder_path'] = property_folder  # Update the CSV
                        image_processed = True

                    # Save a screenshot of the loaded page
                    screenshot_path = os.path.join(property_folder, f"{property_id}_picture_{i}_screenshot.jpg")
                    driver.save_screenshot(screenshot_path)
                    print(f"Saved screenshot for {property_id}_picture_{i}")

                    # Crop the screenshot to remove black spaces
                    cropped_path = os.path.join(property_folder, f"{property_id}_picture_{i}.jpg")
                    crop_screenshot(screenshot_path, cropped_path)

                    # Remove the original un-cropped screenshot
                    os.remove(screenshot_path)
                except Exception as e:
                    print(f"Error opening {image_url}: {e}")

    # Quit the driver
    driver.quit()

    # Save the updated DataFrame to the final CSV
    df.to_csv(output_csv, index=False)
    print(f"Final CSV saved to {output_csv}")


# Example usage
input_csv = "/Users/shubhamagarwal/Documents/Northeastern/Semester_3/final_project/Data/properties_step2/added_propertyid/merged_properties.csv"  # Replace with the path to your updated CSV
output_csv = "/Users/shubhamagarwal/Documents/Northeastern/Semester_3/final_project/Data/properties_step3/properties_step3.csv"  # Replace with the path to save the final CSV
output_folder = "/Users/shubhamagarwal/Documents/Northeastern/Semester_3/final_project/Data/properties_step3/property_images"  # Folder to save property images

user_agent = (
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/113.0.5672.126 Safari/537.36"
)

open_images_in_browser(input_csv, output_csv, output_folder, user_agent)


/opt/anaconda3/lib/python3.12/site-packages/selenium/webdriver/remote/webdriver.py:829: UserWarning: name used for saved screenshot does not match file type. It should end with a `.png` extension
  return self.get_screenshot_as_file(filename)


Saved screenshot for property_1_picture_1
Cropped and saved image to /Users/shubhamagarwal/Documents/Northeastern/Semester_3/final_project/Data/properties_step3/property_images/property_1/property_1_picture_1.jpg
Saved screenshot for property_1_picture_2
Cropped and saved image to /Users/shubhamagarwal/Documents/Northeastern/Semester_3/final_project/Data/properties_step3/property_images/property_1/property_1_picture_2.jpg
Saved screenshot for property_1_picture_3
Cropped and saved image to /Users/shubhamagarwal/Documents/Northeastern/Semester_3/final_project/Data/properties_step3/property_images/property_1/property_1_picture_3.jpg
Saved screenshot for property_1_picture_4
Cropped and saved image to /Users/shubhamagarwal/Documents/Northeastern/Semester_3/final_project/Data/properties_step3/property_images/property_1/property_1_picture_4.jpg
Saved screenshot for property_1_picture_5
Cropped and saved image to /Users/shubhamagarwal/Documents/Northeastern/Semester_3/final_project/Data/prop

In [71]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

def get_driver():

    options = Options()
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.126 Safari/537.36"
    )
    options.add_argument("--start-maximized")  # Start browser maximized
    driver = webdriver.Chrome(options=options)
    return driver

def scrape_property_details(csv_file, output_csv, start_row=0, end_row=None):

    # Read the input CSV
    properties = pd.read_csv(csv_file)
    
    # Apply slicing based on start_row and end_row
    properties = properties.iloc[start_row:end_row]

    # Initialize new columns for scraped data
    properties['Education'] = None
    properties['Transportation'] = None
    properties['Feasibility'] = None
    properties['Point_of_Interest'] = None
    properties['Additional_Property_Details'] = None

    driver = get_driver()

    for index, row in properties.iterrows():
        property_url = row['property_url']
        if pd.isna(property_url):
            continue

        try:
            # Open the property URL
            driver.get(property_url)
            time.sleep(3)  # Allow page to load

            # Scrape data for each tag
            education = ""
            transportation = ""
            feasibility = ""
            point_of_interest = ""

            try:
                education = driver.find_element(By.XPATH, '//*[@id="schoolsSection"]').text
            except Exception as e:
                print(f"Error scraping Education for row {index}: {e}")

            try:
                transportation = driver.find_element(By.XPATH, '//*[@id="transportationSection"]').text
            except Exception as e:
                print(f"Error scraping Transportation for row {index}: {e}")

            try:
                feasibility = driver.find_element(By.XPATH, '//*[@id="walkScoreSection"]').text
            except Exception as e:
                print(f"Error scraping Feasibility for row {index}: {e}")

            try:
                point_of_interest = driver.find_element(By.XPATH, '//*[@id="nearbyAmenitiesSection"]').text
            except Exception as e:
                print(f"Error scraping Point of Interest for row {index}: {e}")

            # Update DataFrame with individual columns
            properties.at[index, 'Education'] = education
            properties.at[index, 'Transportation'] = transportation
            properties.at[index, 'Feasibility'] = feasibility
            properties.at[index, 'Point_of_Interest'] = point_of_interest

            # Combine all columns into Additional_Property_Details
            combined_details = "\n\n".join(filter(None, [education, transportation, feasibility, point_of_interest]))
            properties.at[index, 'Additional_Property_Details'] = combined_details

            print(f"Scraped data for row {index}")

        except Exception as e:
            print(f"Error processing row {index}: {e}")

    # Quit the WebDriver
    driver.quit()

    # Save the updated DataFrame to a new CSV
    properties.to_csv(output_csv, index=False)
    print(f"Scraped data saved to {output_csv}")

# Example usage
input_csv = "/Users/shubhamagarwal/Documents/Northeastern/Semester_3/final_project/Data/properties_step3/properties_step3.csv"  # Input CSV from step 3
output_csv = "/Users/shubhamagarwal/Documents/Northeastern/Semester_3/final_project/Data/properties_step4/properties_step4.csv"  # Path to save the updated CSV

scrape_property_details(input_csv, output_csv, start_row=0, end_row=400)  # Adjust start_row and end_row as needed


Scraped data for row 0
Scraped data for row 1
Scraped data for row 2
Scraped data for row 3
Scraped data for row 4
Scraped data for row 5
Scraped data for row 6
Scraped data for row 7
Scraped data for row 8
Scraped data for row 9
Scraped data for row 10
Scraped data for row 11
Scraped data for row 12
Scraped data for row 13
Scraped data for row 14
Scraped data for row 15
Scraped data for row 16
Scraped data for row 17
Scraped data for row 18
Scraped data for row 19
Scraped data for row 20
Scraped data for row 21
Scraped data for row 22
Scraped data for row 23
Scraped data for row 24
Scraped data for row 25
Scraped data for row 26
Scraped data for row 27
Scraped data for row 28
Scraped data for row 29
Scraped data for row 30
Scraped data for row 31
Scraped data for row 32
Scraped data for row 33
Scraped data for row 34
Scraped data for row 35
Scraped data for row 36
Scraped data for row 37
Scraped data for row 38
Scraped data for row 39
Scraped data for row 40
Scraped data for row 41
Sc